In [1]:
!pip install contractions
import warnings
warnings.filterwarnings('ignore')
import statistics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_digits
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
! pip install bs4  # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz



[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jason\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
ERROR: Invalid requirement: '#'

[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
df = pd.read_csv('train.csv', sep=',',error_bad_lines=False)
df = df[['full_text','cohesion']]
df['star_rating'] = pd.to_numeric(df['cohesion'], errors='coerce')
df = df[['full_text','star_rating']]
#df['star_rating'] = df['star_rating'].astype(str)
rev_and_rating = df.rename({'full_text': 'review_body'}, axis=1)

In [3]:

type(df['star_rating'].head(1).values[0])

numpy.float64

In [4]:
rev_and_rating

,review_body,star_rating
0,I think that students would benefit from learn...,3.5
1,When a problem is a change you have to let it ...,2.5
2,"Dear, Principal\n\nIf u change the school poli...",3.0
3,The best time in life is when you become yours...,4.5
4,Small act of kindness can impact in other peop...,2.5
...,...,...
3906,I believe using cellphones in class for educat...,2.5
3907,"Working alone, students do not have to argue w...",4.0
3908,"""A problem is a chance for you to do your best...",2.5
3909,Many people disagree with Albert Schweitzer's ...,4.0


In [5]:
rev_and_rating.star_rating

0       3.5
1       2.5
2       3.0
3       4.5
4       2.5
       ... 
3906    2.5
3907    4.0
3908    2.5
3909    4.0
3910    3.5
Name: star_rating, Length: 3911, dtype: float64

In [6]:
rev_and_rating.shape

(3911, 2)

In [7]:
balanced_data = rev_and_rating

# Data Cleaning



In [8]:
#print the average length
before_clean_len = balanced_data['review_body'].astype(str).apply(len).mean()

convert the all reviews into the lower case

In [9]:
balanced_data

,review_body,star_rating
0,I think that students would benefit from learn...,3.5
1,When a problem is a change you have to let it ...,2.5
2,"Dear, Principal\n\nIf u change the school poli...",3.0
3,The best time in life is when you become yours...,4.5
4,Small act of kindness can impact in other peop...,2.5
...,...,...
3906,I believe using cellphones in class for educat...,2.5
3907,"Working alone, students do not have to argue w...",4.0
3908,"""A problem is a chance for you to do your best...",2.5
3909,Many people disagree with Albert Schweitzer's ...,4.0


In [10]:
balanced_data['review_body'] = balanced_data['review_body'].str.lower()

In [11]:
balanced_data

,review_body,star_rating
0,i think that students would benefit from learn...,3.5
1,when a problem is a change you have to let it ...,2.5
2,"dear, principal\n\nif u change the school poli...",3.0
3,the best time in life is when you become yours...,4.5
4,small act of kindness can impact in other peop...,2.5
...,...,...
3906,i believe using cellphones in class for educat...,2.5
3907,"working alone, students do not have to argue w...",4.0
3908,"""a problem is a chance for you to do your best...",2.5
3909,many people disagree with albert schweitzer's ...,4.0


change contraction to long form

In [12]:
import contractions
balanced_data['review_body']=balanced_data['review_body'].apply(lambda x: contractions.fix(str(x)))

In [13]:
balanced_data

,review_body,star_rating
0,i think that students would benefit from learn...,3.5
1,when a problem is a change you have to let it ...,2.5
2,"dear, principal\n\nif you change the school po...",3.0
3,the best time in life is when you become yours...,4.5
4,small act of kindness can impact in other peop...,2.5
...,...,...
3906,i believe using cellphones in class for educat...,2.5
3907,"working alone, students do not have to argue w...",4.0
3908,"""a problem is a chance for you to do your best...",2.5
3909,many people disagree with albert schweitzer's ...,4.0


remove URLs

In [14]:
import re
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
balanced_data

,review_body,star_rating
0,i think that students would benefit from learn...,3.5
1,when a problem is a change you have to let it ...,2.5
2,"dear, principal\n\nif you change the school po...",3.0
3,the best time in life is when you become yours...,4.5
4,small act of kindness can impact in other peop...,2.5
...,...,...
3906,i believe using cellphones in class for educat...,2.5
3907,"working alone, students do not have to argue w...",4.0
3908,"""a problem is a chance for you to do your best...",2.5
3909,many people disagree with albert schweitzer's ...,4.0


In [15]:
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: re.sub('<[^<]+?>', '', str(x)))
balanced_data

,review_body,star_rating
0,i think that students would benefit from learn...,3.5
1,when a problem is a change you have to let it ...,2.5
2,"dear, principal\n\nif you change the school po...",3.0
3,the best time in life is when you become yours...,4.5
4,small act of kindness can impact in other peop...,2.5
...,...,...
3906,i believe using cellphones in class for educat...,2.5
3907,"working alone, students do not have to argue w...",4.0
3908,"""a problem is a chance for you to do your best...",2.5
3909,many people disagree with albert schweitzer's ...,4.0


remove non-alphabetical characters

remove extra spaces

In [16]:
balanced_data['review_body'] = balanced_data['review_body'].str.replace('[^a-zA-Z ]', '')
balanced_data

,review_body,star_rating
0,i think that students would benefit from learn...,3.5
1,when a problem is a change you have to let it ...,2.5
2,dear principalif you change the school policy ...,3.0
3,the best time in life is when you become yours...,4.5
4,small act of kindness can impact in other peop...,2.5
...,...,...
3906,i believe using cellphones in class for educat...,2.5
3907,working alone students do not have to argue wi...,4.0
3908,a problem is a chance for you to do your best ...,2.5
3909,many people disagree with albert schweitzers q...,4.0


In [17]:
#print the average length
after_clean_len = balanced_data['review_body'].astype(str).apply(len).mean()
print(before_clean_len,' , ', after_clean_len)

2334.522372794682  ,  2296.104321145487


# Pre-processing

## remove the stop words 

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
stop_words = stopwords.words('english')
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [20]:
balanced_data[balanced_data.star_rating==1]

,review_body,star_rating
39,name genericname story carsi bay car new live ...,1.0
952,student offer distance learning option student...,1.0
1161,people value self defrine indidual follow yea ...,1.0
1540,good prescient us procession prescient us many...,1.0
2989,good thinkingbut see fast fallsaid ageeri said...,1.0
3037,people value ideas came millonarie person exam...,1.0
3086,technology positive good possibility people ac...,1.0
3342,yes agrey lifes important people value selfrel...,1.0
3679,memorable teacher ever hadwas teacher wordwild...,1.0
3706,bring phones school school phones get get scho...,1.0


## perform lemmatization  

:param pos: The Part Of Speech tag. Valid options are `"n"` for nouns,
            `"v"` for verbs, `"a"` for adjectives, `"r"` for adverbs and `"s"`
            for satellite adjectives.

In [21]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos ="a") for word in x.split()]))
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos ="v") for word in x.split()]))
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos ="r") for word in x.split()]))
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos ="s") for word in x.split()]))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jason\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [22]:
balanced_data = balanced_data.reset_index()
del balanced_data['index']

In [23]:
balanced_data.star_rating.value_counts()

3.0    1096
3.5     988
2.5     790
4.0     534
2.0     315
4.5     125
1.5      27
5.0      26
1.0      10
Name: star_rating, dtype: int64

In [24]:
#print the average length
after_prepro_len = balanced_data['review_body'].astype(str).apply(len).mean()
print(after_clean_len,' , ',after_prepro_len)

2296.104321145487  ,  1261.80542060854


In [25]:
X = balanced_data['review_body']
y = balanced_data['star_rating']

# TF-IDF Feature Extraction

In [26]:
tfIdfVectorizer1=TfidfVectorizer(max_features=1000)
tfIdf_x = tfIdfVectorizer1.fit_transform(X)
tfIdf_x_array = tfIdf_x.toarray()
#feature_names = tfIdfVectorizer1.get_feature_names()
#tfIdf_x_df = pd.DataFrame(tfIdf_x_array, columns = feature_names)

# divide train and test

In [27]:
train_x, test_x, train_y, test_y = train_test_split(tfIdf_x_array, y, test_size=0.2, random_state=42)

In [28]:
test_x

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.20069227, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [49]:
from collections import Counter
print(Counter(train_y))

Counter({'3.0': 881, '3.5': 769, '2.5': 639, '4.0': 431, '2.0': 253, '4.5': 106, '5.0': 21, '1.5': 20, '1.0': 8})


In [30]:
def printstats(test_y, pred_y):
    precision_list = precision_score(test_y,pred_y,average = None)
    recall_list = recall_score(test_y,pred_y,average = None)
    f1_list = f1_score(test_y, pred_y, average=None)
    #class1:
    
    for i in range(1,6):
        print(f'{precision_list[i-1]},{recall_list[i-1]},{f1_list[i-1]}')
    print(f"{statistics.mean(precision_list)},{statistics.mean(recall_list)},{f1_score(test_y, pred_y, average='weighted')}")
    #print(f1_score(test_y, pred_y, average=None))


# Perceptron

In [31]:
train_y = train_y.astype(str)
test_y = test_y.astype(str)

In [32]:
percept_clf = Perceptron()
percept_clf.fit(train_x, train_y)
perc_pred_y = percept_clf.predict(test_x)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, perc_pred_y)

0.227330779054917

In [34]:
perc_f1 = f1_score(test_y, perc_pred_y, average='macro')
perc_f1

0.13709970338462352

In [35]:
precision_score(test_y,perc_pred_y,average = None)

array([0.03846154, 0.        , 0.18518519, 0.24423963, 0.31132075,
       0.25506073, 0.19090909, 0.07692308, 0.        ])

In [36]:
printstats(test_y, perc_pred_y)

0.038461538461538464,0.5,0.07142857142857144
0.0,0.0,0.0
0.18518518518518517,0.08064516129032258,0.11235955056179775
0.24423963133640553,0.3509933774834437,0.2880434782608696
0.3113207547169811,0.15348837209302327,0.205607476635514
0.14467777847524638,0.18688275519820774,0.2248054435379464


# SVM

In [37]:
#z = '''
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


svm_clf = SVC(kernel="linear")

#'''


In [38]:
svm_clf.fit(train_x, train_y)


SVC(kernel='linear')

In [39]:
#z = '''
svm_pred_y = svm_clf.predict(test_x)
svm_f1 = f1_score(test_y, svm_pred_y, average='macro')

#'''

In [40]:
printstats(test_y, svm_pred_y)

0.0,0.0,0.0
0.0,0.0,0.0
0.0,0.0,0.0
0.25196850393700787,0.2119205298013245,0.23021582733812954
0.28184281842818426,0.48372093023255813,0.35616438356164387
0.11981619281340641,0.1255613404756557,0.25021755422589215


# Logistic Regression

In [41]:
from sklearn.linear_model import LogisticRegression
logi_clf = LogisticRegression(random_state=0).fit(train_x, train_y)
logi_pred_y = logi_clf.predict(test_x)

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, logi_pred_y)

0.2784163473818646

In [43]:
logi_f1 = f1_score(test_y, logi_pred_y, average='macro')
logi_f1

0.11677922419244863

In [44]:
printstats(test_y, logi_pred_y)

0.0,0.0,0.0
0.0,0.0,0.0
0.0,0.0,0.0
0.23529411764705882,0.2119205298013245,0.2229965156794425
0.2756598240469208,0.4372093023255814,0.3381294964028777
0.11835051085025193,0.1250876624811545,0.2511395102147756


# Naive Bayes

In [45]:
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB().fit(train_x, train_y)
gnb_pred_y = gnb_clf.predict(test_x)

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, gnb_pred_y)

0.15197956577266922

In [47]:
nb_f1 = f1_score(test_y, gnb_pred_y, average='macro')
nb_f1

0.10065552199125632

In [48]:
printstats(test_y, gnb_pred_y)

0.0,0.0,0.0
0.0,0.0,0.0
0.09574468085106383,0.2903225806451613,0.144
0.19101123595505617,0.11258278145695365,0.1416666666666667
0.2605042016806723,0.14418604651162792,0.18562874251497008
0.12697942575446283,0.14545687045136174,0.16618107664042892
